In [1]:
import probtorch
import torch

import combinators
import hmm
import smc
import utils

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [4]:
for t in range(zs.shape[1] - 1):
    zs, mu, sigma, delta, generative = hmm.hmm_step(zs, mu, sigma, delta, t+1, generative)

In [5]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [6]:
num_particles = 500

In [7]:
smc_step = smc.smc(hmm.hmm_step, hmm.hmm_retrace)

In [8]:
hmm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [9]:
inference, init_hmm = smc.variational_smc(num_particles, hmm.init_hmm, smc_step, 500, 50, hmm_params, generative, 5)

In [10]:
inference

Trace{'mu': Normal([torch.FloatTensor of size 500]), 'sigma': Normal([torch.FloatTensor of size 500]), 'delta': Normal([torch.FloatTensor of size 500]), 'Z_0': Normal([torch.FloatTensor of size 500]), 'Z_1': Normal([torch.FloatTensor of size 500]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 500]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 500]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 500]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 500]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 500]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 500]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 500]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 500]),

In [11]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [12]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - generative[key].value)**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.001479
SMC MSE at time 2: 30.054335
SMC MSE at time 3: 22.487921
SMC MSE at time 4: 7.175554
SMC MSE at time 5: 20.824797
SMC MSE at time 6: 18.373857
SMC MSE at time 7: 5.069932
SMC MSE at time 8: 2.358100
SMC MSE at time 9: 0.422409
SMC MSE at time 10: 15.750609
SMC MSE at time 11: 2.560673
SMC MSE at time 12: 3.379993
SMC MSE at time 13: 18.762814
SMC MSE at time 14: 0.001609
SMC MSE at time 15: 0.000003
SMC MSE at time 16: 0.087420
SMC MSE at time 17: 4.336171
SMC MSE at time 18: 0.277719
SMC MSE at time 19: 2.539819
SMC MSE at time 20: 3.632300
SMC MSE at time 21: 0.555960
SMC MSE at time 22: 0.048985
SMC MSE at time 23: 0.053555
SMC MSE at time 24: 11.865206
SMC MSE at time 25: 4.122310
SMC MSE at time 26: 3.180465
SMC MSE at time 27: 1.965402
SMC MSE at time 28: 1.483395
SMC MSE at time 29: 5.827371
SMC MSE at time 30: 0.833220
SMC MSE at time 31: 5.546090
SMC MSE at time 32: 6.011144
SMC MSE at time 33: 1.046634
SMC MSE at time 34: 6.954966
SMC MSE at time 